## 🔶 Leitura dos Arquivos no DBFS

In [0]:
# Ler os dados do DBFS em vez do S3
df_estatisticas = spark.read.parquet("dbfs:/mnt/silver_temp/estatisticas")

df_estatisticas.createOrReplaceTempView("estatisticas")

print('✅ Dataframes criado com sucesso a partir do DBFS')

✅ Dataframes criado com sucesso a partir do DBFS


### 🔹Explodir as posições em campo


In [0]:
%sql
-- Explode as posições separadas por vírgula
CREATE OR REPLACE TEMP VIEW posicoes_explodidas AS
SELECT 
    Jogador,
    Clube,
    Numero_Camisa,
    Nacionalidade,
    Data,
    TRIM(posicao) AS Posicao
FROM (
    SELECT *,
        EXPLODE(SPLIT(Posicao_em_Campo, ',')) AS posicao
    FROM estatisticas
);


### 🔹Contar frequência das posições por jogador


In [0]:
%sql
-- Atualizado para considerar Clube
CREATE OR REPLACE TEMP VIEW frequencia_posicoes AS
SELECT 
    Jogador,
    Clube,
    Nacionalidade,
    Posicao,
    COUNT(*) AS frequencia
FROM posicoes_explodidas
GROUP BY Jogador, Clube, Nacionalidade, Posicao;


### 🔹Aplicar ranking (posição mais jogada = rank 1)

In [0]:
%sql
-- Agora o ranking é por Jogador + Clube
CREATE OR REPLACE TEMP VIEW ranking_posicoes AS
SELECT *,
    ROW_NUMBER() OVER (
        PARTITION BY Jogador, Clube
        ORDER BY frequencia DESC
    ) AS pos_rank
FROM frequencia_posicoes;


### 🔹Selecionar posição principal


In [0]:
%sql
-- Principal
CREATE OR REPLACE TEMP VIEW principal AS
SELECT 
    Jogador,
    Clube,
    Nacionalidade,
    Posicao AS Posicao_Principal,
    frequencia AS jogos_posicao_principal
FROM ranking_posicoes
WHERE pos_rank = 1;

### 🔹Selecionar improvisação (segunda mais jogada)


In [0]:
%sql
-- Improvisação
CREATE OR REPLACE TEMP VIEW improvisacao AS
SELECT 
    Jogador,
    Clube,
    Posicao AS Improvisacao,
    frequencia AS jogos_improvisados
FROM ranking_posicoes
WHERE pos_rank = 2;

### 🔹Aplicar um JOIN nas duas tabelas (Posição Principal e Improvisação)


In [0]:
%sql
CREATE OR REPLACE TEMP VIEW Mart_Info_Jogadores AS
SELECT 
    p.Jogador,
    p.Clube,
    p.Nacionalidade,
    
    p.Posicao_Principal,
    CASE p.Posicao_Principal
        WHEN 'CB' THEN 'Zagueiro'
        WHEN 'FW' THEN 'Atacante'
        WHEN 'GK' THEN 'Goleiro'
        WHEN 'RW' THEN 'Ponta Direita'
        WHEN 'RB' THEN 'Lateral Direito'
        WHEN 'CM' THEN 'Meio-Campista Central'
        WHEN 'AM' THEN 'Meia Ofensivo'
        WHEN 'LB' THEN 'Lateral Esquerdo'
        WHEN 'LM' THEN 'Meia Esquerda'
        WHEN 'LW' THEN 'Ponta Esquerda'
        WHEN 'DM' THEN 'Volante'
        WHEN 'WB' THEN 'Ala'
        WHEN 'RM' THEN 'Meia Direita'
        ELSE NULL
    END AS Nome_Posicao_Principal,
    
    p.jogos_posicao_principal,

    i.Improvisacao,
    CASE i.Improvisacao
        WHEN 'CB' THEN 'Zagueiro'
        WHEN 'FW' THEN 'Atacante'
        WHEN 'GK' THEN 'Goleiro'
        WHEN 'RW' THEN 'Ponta Direita'
        WHEN 'RB' THEN 'Lateral Direito'
        WHEN 'CM' THEN 'Meio-Campista Central'
        WHEN 'AM' THEN 'Meia Ofensivo'
        WHEN 'LB' THEN 'Lateral Esquerdo'
        WHEN 'LM' THEN 'Meia Esquerda'
        WHEN 'LW' THEN 'Ponta Esquerda'
        WHEN 'DM' THEN 'Volante'
        WHEN 'WB' THEN 'Ala'
        WHEN 'RM' THEN 'Meia Direita'
        ELSE NULL
    END AS Nome_Improvisacao,

    i.jogos_improvisados

FROM principal p
LEFT JOIN improvisacao i
  ON p.Jogador = i.Jogador AND p.Clube = i.Clube
ORDER BY p.Jogador;


## 🔶 Conferencias finais

### 🔹Conferindo se temos Mais de 1 jogador com o mesmo nome no mesmo clube

In [0]:
%sql
-- Combinações únicas de jogador e clube (removendo duplicatas)
SELECT 
    Jogador,
    Clube,
    COUNT(*) AS ocorrencias
FROM (
    SELECT DISTINCT Jogador, Clube
    FROM estatisticas
) AS unicos
GROUP BY Jogador, Clube
ORDER BY Jogador;


Jogador,Clube,ocorrencias
Abner,Juventude,1
Ademir Santos,Bahia,1
Adriano Martins,Atlético-GO,1
Adriel,Bahia,1
Adryelson,Botafogo,1
Adson,Vasco da Gama,1
Agustín Canobbio,Athletico-PR,1
Agustín Giay,Palmeiras,1
Agustín Marchesín,Grêmio,1
Agustín Rogel,Internacional,1


### 🔹Conferindo se temos Jogadores com mesmo nome no mesmo clube, mas camisas diferentes

In [0]:
%sql
-- Jogadores com mesmo nome no mesmo clube, mas camisas diferentes
SELECT 
    Jogador,
    Clube,
    COUNT(DISTINCT Numero_Camisa) AS qtd_camisas_diferentes
FROM estatisticas
GROUP BY Jogador, Clube
HAVING COUNT(DISTINCT Numero_Camisa) > 1
ORDER BY qtd_camisas_diferentes DESC;


Jogador,Clube,qtd_camisas_diferentes
Alejo Cruz,Atlético-GO,10
Budiga Roni,Atlético-GO,7
Rhaldney,Atlético-GO,6
Matías Lacava,Atlético-GO,6
Lucas Kal,Atlético-GO,6
Joel Campbell,Atlético-GO,6
Shaylon,Atlético-GO,6
Derek,Atlético-GO,6
Luiz Felipe,Atlético-GO,6
Janderson,Atlético-GO,5


## 📌 Salvando os Arquivs na Camada Gold 

In [0]:
df_gold = spark.sql("SELECT * FROM Mart_Info_Jogadores")

df_gold.write.mode("overwrite").parquet("s3a://mvp-brasileirao-2024/gold/Mart_Info_Jogadores")